In [3]:
import numpy as np, pandas as pd, json, glob, os, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
# import shutil
from sentence_transformers import SentenceTransformer



In [4]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


In [ ]:
client = QdrantClient(path="../data/qdrant_local")  # 데이터는 ./qdrant_local 폴더에 저장됨

# 2️⃣ 컬렉션 이름 및 파라미터 정의
collection_name = "TL_내과"

In [27]:
client.delete_collection(collection_name=collection_name)

True

In [20]:
# client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
model = SentenceTransformer("BAAI/bge-m3")


In [28]:
path = '../../../data/1.데이터/Training/02.라벨링데이터/'
os.listdir(path)
for name_ in os.listdir(path):
    print(name_)
    collection_name = name_
    client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
    for i in tqdm(os.listdir(path+name_)):
        with open(path+name_+'/'+i, "r", encoding="utf-8-sig") as f:
            data = json.load(f)
        
        
        embedding = model.encode(data["question"]).tolist()
        
        client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=data["qa_id"],
                vector=embedding,
                payload={
                    "domain": data["domain"],
                    "q_type": data["q_type"],
                    "question": data["question"],
                    "answer": data["answer"],
                    },
                )
            ],
        )
    print(f"{name_} done===============")
        
        
        
        
        
        
    #     break
    # break        

TL_내과


  0%|          | 0/10299 [00:00<?, ?it/s]

100%|██████████| 10299/10299 [09:56<00:00, 17.28it/s]


TL_내과 done===============
TL_산부인과


100%|██████████| 1991/1991 [01:53<00:00, 17.48it/s]


TL_산부인과 done===============
TL_소아청소년과


100%|██████████| 2421/2421 [02:13<00:00, 18.12it/s]


TL_소아청소년과 done===============
TL_응급의학과


100%|██████████| 649/649 [00:27<00:00, 23.68it/s]

TL_응급의학과 done===============


'1) 생활 습관 개선 교육 및 상담'

In [ ]:
# 3️⃣ 임베딩 모델 로드 (bge-m3)
model = SentenceTransformer("BAAI/bge-m3")


# 5️⃣ question 임베딩 생성
# embedding = model.encode(data["question"]).tolist()

# 6️⃣ Qdrant에 데이터 업로드
client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(
            id=data["qa_id"],
            vector=embedding,
            payload={
                "domain": data["domain"],
                "q_type": data["q_type"],
                "question": data["question"],
                "answer": data["answer"],
            },
        )
    ],
)

print("✅ 데이터 업로드 완료!")

✅ 데이터 업로드 완료!


In [5]:
if collection_name not in [c.name for c in client.get_collections().collections]:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
    print(f"✅ 컬렉션 '{collection_name}' 생성 완료")
else:
    print(f"ℹ️ 컬렉션 '{collection_name}' 이미 존재")

ℹ️ 컬렉션 'medical_qa' 이미 존재
